In [1]:
#=======================================================================================
# sentencepiece 사전 모델 처음부터 만드는 예제
# => 말뭉치를 가지고, sp vocab을 만듬(학습)
#
# [인자]
# -input_file : 입력할 말뭉치 데이터 경로 지정
# -vocab_size : BPE의 단어수를 얼마로 할 것인가 이다. 너무 적으면 한 글자 단위로 쪼개지는 경향이 있고, 너무 많으면 쓸데없는 단어들이 만들어진다. 
#               주로 3,2000이 가장 좋다고 알려져 있다.
# -model_name : 저장할 이름이다. 학습하고 나면 .model, .vocab 2개의 파일이 만들어진다.
# -model_type : bpe, unigram 등이 있는데 두가지를 모두 사용해 보고 성능이 좋은 거로 고르도록 하자.
# -character_coverage : 모든 단어를 커버할것인가, 너무 희귀한 단어는 뺄 것인가 이다. 학습 코퍼스가 대용량이라면 보통 0.9995로 사용하면 된다. 그런데 코퍼스가 작다면 1.0으로 지정하자. 그럼 [UNK]가 없다.
# -user_defined_symbols : BPE로 생성된 단어 외 알고리즘에서 사용할 특수문자들을 지정한다. BERT 기반 요약, 번역모델 개발 시 BOS, EOS, SEP 등 추가 토큰들이 필요하다. 
#                         그러므로 user_defined_symbols을 이용해 Dummy token(UNK, unused, BOS, EOS 등등)을 추가해준다. 
#                         HTTP나 network traffic에 BPE를 적용해야 할 경우가 있다. 
#                         이때 user_defined_symbols 인자에 'http, GET, POST, User-Agent 등 HTTP 데이터에서 자주 나오는 Token들을 지정하기도 했다.
# vocab_size에 비례하여 학습시간이 결정된다. 필자는 32,000으로 지정하여 노트북에서 대략 43sec가 소요됐다.
#
# sentencepiecr 설치
#!pip install sentencepiece
#=======================================================================================
import sentencepiece as spm
import os

In [2]:
input_file = '../../../AI/kopora/kowiki_20190620_Full/wiki_20190620.txt'
vocab_size=32000

# 모델 root 경로 지정
model_root = '../../sp/'
if not os.path.isdir(model_root):
    os.mkdir(model_root)
    
# 모델명과, 풀경로 지정    
model_name = 'spm_{}_token'.format(vocab_size)
model_fullpath = os.path.join(model_root, model_name)
print(model_fullpath)

model_type = 'bpe'
character_coverage = 0.9995
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS]'


../../sp/spm_32000_token


In [3]:
# 훈련 파라메터 설정
input_argument = '--input={0}\
 --model_prefix={1}\
 --vocab_size={2}\
 --user_defined_symbols={3}\
 --model_type={4}\
 --character_coverage={5}'\
.format(input_file, model_fullpath, vocab_size, user_defined_symbols, model_type, character_coverage)

print(input_argument)

--input=../../../AI/kopora/kowiki_20190620_Full/wiki_20190620.txt --model_prefix=../../sp/spm_32000_token --vocab_size=32000 --user_defined_symbols=[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS] --model_type=bpe --character_coverage=0.9995


In [4]:
# 훈련 시작
# => 훈련이 완료되고 나면 해당경로에 xxx.model, xxx.vocab 2개의 파일이 생성된다. (오래 걸림)
spm.SentencePieceTrainer.Train(input_argument)

In [5]:
# 검증 시작
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(model_fullpath))

tokens = sp.encode_as_pieces('나는 아침에 배를 먹었다')
ids = sp.encode_as_ids('나는 아침에 배를 먹었다')

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[3906, 19469, 8434, 1046, 19]
['▁나는', '▁아침에', '▁배를', '▁먹', '었다']
나는 아침에 배를 먹었다
나는 아침에 배를 먹었다
